In [1]:
from pyspark.sql import SparkSession

spark = (SparkSession.builder
         .appName("write-data")
         .master("spark://spark-master:7077")
         .config("spark.executor.memory", "512m")
         .getOrCreate())

spark.sparkContext.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/19 12:19:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType

df = (spark.read.format("csv")
      .option("header", "true")
      .option("nullValue", "null")
      .option("dateFormat", "LLLL d, y")
      .load("../data/netflix_titles.csv"))

In [7]:
(df.write.format("csv")
 .option("header", "true") 
 .mode("overwrite")
 .option("delimiter", ",")
 .save("../data/data_lake/netflix_csv_data"))

In [8]:
(df.write.format("json") 
 .mode("overwrite") 
 .save("../data/data_lake/netflix_json_data"))

In [9]:
(df.write.format("parquet") 
 .mode("overwrite") 
 .save("../data/data_lake/netflix_parquet_data"))

### Write Compressed Data

In [12]:
(df.write
 .format("csv")
 .mode("overwrite") 
 .option("header", "true")
 .option("delimiter", ",")
 .option("codec", "org.apache.hadoop.io.compress.GzipCodec")
 .save("../data/data_lake/netflix_csv_data.gz"))

### Specify the Number of Partitions

In [14]:
(df.repartition(4) 
 .write.format("csv") 
 .mode("overwrite") 
 .option("header", "true") 
 .option("delimiter", ",") 
 .save("../data/data_lake/netflix_csv_data_4_part")) 

### Use `coalesce()` to Reduce number fo Partitions

In [15]:
(df.coalesce(1)
 .write.format("csv")
 .mode("overwrite") 
 .option("header", "true")
 .option("delimiter", ",")
 .save("../data/data_lake/netflix_csv_data_whole"))

### Use `partitionBy()` to write partitions based on a column

In [16]:
# partition the CSV data by the 'release_year' column
(df.write.format('csv')
 .option('header', 'true')
 .option('delimiter', ',')
 .mode('overwrite')
 .partitionBy('release_year')
 .save("../data/data_lake/netflix_csv_data_partitioned"))

In [17]:
spark.stop()